<a href="https://colab.research.google.com/github/wtglad/signed-area-causal-inference/blob/main/Signed_Area_Causal_Discovery_Example_Workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install signatory==1.2.4.1.7.1 --no-cache-dir --force-reinstall

     |████████████████████████████████| 61 kB 9.4 MB/s 
  Created wheel for signatory: filename=signatory-1.2.4.1.7.1-cp37-cp37m-linux_x86_64.whl size=7533684 sha256=e407fe557d681c22e7e8a7bda481b4d3bbcc4972ccaddea3a8badd7d778f9a88
  Stored in directory: /tmp/pip-ephem-wheel-cache-1_7lnp8h/wheels/fb/2d/23/eebc83e1c8668c2193faf4e9eb66a34c8357b9ab9cfc49775d
Successfully built signatory


In [2]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import signatory
from itertools import product
import torch
from sklearn.preprocessing import MinMaxScaler
import scipy.stats
from progressbar import ProgressBar

# Helper functions

In [3]:
def preprocess_df(df, features):

  # Scale features by setting min/max difference to 1 and centering to zero mean

  scaler = MinMaxScaler()
  df[features] = scaler.fit_transform(df[features])
  
  for i in features:
    df[i] = df[i] - df[i].mean()

  df = df.dropna()
  df = df.reset_index(drop=True)
  df['t'] = df.index
  
  return df


In [4]:
def conf_seq(t, alpha, mu, sigma, rho=1):
  
  pm = np.array([+1, -1])
  
  conf_seq_ub, conf_seq_lb = mu + ((pm * sigma) * np.sqrt(((2 * (t * (rho ** 2) + 1))/((t ** 2) * (rho ** 2))) *
                                                          np.log((np.sqrt(t * (rho ** 2) + 1))/(alpha / 2))))
  
  return conf_seq_ub, conf_seq_lb

In [5]:
def check_in_bounds(signed_area_value, upper_bound, lower_bound):
  if signed_area_value > upper_bound:
    return 1
  elif signed_area_value < lower_bound:
    return -1
  else:
    return 0

In [6]:

def shuffled_signed_area_test(df, path_length, features, n_shuffles, plot_figures=False):
  
  # Create list of feature pairs
  pairwise_signature_terms = []
  sig_features = features.copy()

  for prod in product(features, features):
    sig_features.append(prod)
    if prod[0] != prod[1]:
      pairwise_signature_terms.append(prod)

  # Compute depth 1 and 2 path signatures and signed areas for original time series 
  data = []

  for i in df.index[path_length:]:
      path_subset = df.iloc[i-path_length:i]
      data.append(path_subset[features].to_numpy())

  depth = 2
  path = torch.Tensor(data)

  signature = signatory.signature(path, depth)
  sig_df = pd.DataFrame(signature.numpy())
  sig_df.columns = ['S' + str(i) for i in sig_features]


  for perm in pairwise_signature_terms:
    sig_df['A(' + perm[0] + ', ' + perm[1] + ')'] = 0.5 * (sig_df["S('" + perm[0] +"', '" + perm[1] + "')"] - sig_df["S('" + perm[1] + "', '" + perm[0] + "')"])

  sig_df = sig_df.merge(df.iloc[path_length:].reset_index(drop=True), left_index=True, right_index=True)

  # Compute signed area for n_shuffles 
  shuffled_signatures = pd.DataFrame()
  
  pbar = ProgressBar()

  for shuffle in pbar(range(n_shuffles)):
    df_shuffled = pd.DataFrame()
    for feature in features:
      df_shuffled[feature] = df[feature].sample(frac=1).reset_index(drop=True)
    
    shuffled_data = []

    for i in df_shuffled.index[path_length:]:
        path_subset = df_shuffled.iloc[i-path_length:i]
        shuffled_data.append(path_subset[features].to_numpy())

    path = torch.Tensor(shuffled_data)
    signature = signatory.signature(path, depth)
    shuffled_sig_df = pd.DataFrame(signature.numpy(), columns =  ['S' + str(i) for i in sig_features])
    shuffled_sig_df['shuffle_index'] = shuffle
    shuffled_sig_df['t'] = df.iloc[path_length:]['t'].reset_index(drop=True)
    

    for perm in pairwise_signature_terms:
        shuffled_sig_df['A(' + perm[0] + ', ' + perm[1] + ')'] = 0.5 * (shuffled_sig_df["S('" + perm[0] + "', '" + perm[1] + "')"] - shuffled_sig_df["S('" + perm[1] + "', '" + perm[0] + "')"])

    shuffled_signatures = shuffled_signatures.append(shuffled_sig_df, ignore_index=True)

  area_columns = [i for i in sig_df.columns.tolist() if 'A(' in i]
  
  # Compute confidence sequences for each pairwise term based on aggregated shuffles
  agg_shuffles = pd.DataFrame()

  for area_col in area_columns: 
    agg_shuffles[area_col] = shuffled_signatures.groupby('t').agg({area_col:'mean'})[area_col]
  
    # Compute cumulative mean and standard deviation and use to create confidence sequence lower and upper bounds
    cumulative_ub = []
    cumulative_lb = []

    for idx in agg_shuffles.index:
      ub, lb = conf_seq(idx, alpha=0.05, mu=agg_shuffles.iloc[:idx][area_col].mean(), sigma=shuffled_sig_df[shuffled_sig_df['t'] <= idx][area_col].std())
  
      cumulative_lb.append(lb)
      cumulative_ub.append(ub)

    agg_shuffles[area_col + '_seq_lb'] = cumulative_lb
    agg_shuffles[area_col + '_seq_ub'] = cumulative_ub

  agg_shuffles = agg_shuffles.reset_index(drop=True)
  sig_df = sig_df.merge(agg_shuffles, suffixes=('', '_shuffled'), left_index=True, right_index=True).dropna(how='any')

  # Compute shuffled signed area deviation at each time point and compute sum for entire series /len(series)
  ssad_summary = []

  for area_col in area_columns: 
    sig_df[area_col + '_SSADt'] = sig_df.apply(lambda x: check_in_bounds(x[area_col], x[area_col + '_seq_ub'], x[area_col + '_seq_lb']), axis=1)
    ssad_summary.append([area_col, sig_df[area_col + '_SSADt'].sum() / len(sig_df)])

  ssad = pd.DataFrame(ssad_summary, columns = ['feature_pair', 'SSAD'])  

  if plot_figures:
    for i in area_columns: 

      plt.figure(figsize=(15,3.5))
      sns.lineplot(x=agg_shuffles.index, y=agg_shuffles[i], label='Shuffled Signed Area $\mu$')
      sns.lineplot(x=sig_df['t'], y=sig_df[i], label='Original Signed Area')
      plt.fill_between(agg_shuffles.index, agg_shuffles[i + '_seq_lb'], agg_shuffles[i + '_seq_ub'], color = 'k', alpha=0.1)
      plt.legend(loc='lower right')
      plt.title(i)
      plt.savefig(i + '.png')
      plt.show()

  return ssad, sig_df, agg_shuffles



# 4 species system

In [7]:
# Data generation process 

y1 = [0.4]
y2 = [0.4]
y3 = [0.4]
y4 = [0.4]


for step in range(999):

  y1_next = y1[-1] * (3.9 - (3.9 * y1[-1]))
  y2_next = y2[-1] * (3.6 - (0.4 * y1[-1]) - (3.6 * y2[-1]))
  y3_next = y3[-1] * (3.6 - (0.4 * y2[-1]) - (3.6 * y3[-1]))
  y4_next = y4[-1] * (3.8 - (0.35 * y3[-1]) - (3.8 * y4[-1]))

  y1.append(y1_next)
  y2.append(y2_next)
  y3.append(y3_next)
  y4.append(y4_next)

In [8]:
df = pd.DataFrame(zip(y1, y2, y3, y4), columns = ['v', 'x', 'y', 'z'])
df['w'] = np.random.randn(len(df))


In [ ]:
# Save output and reload for subsequent iterations if consistency of white noise desired
#df.to_csv('raw_4species.csv', index=False)
#df = pd.read_csv('raw_4species.csv')

In [12]:
df

,v,x,y,z,w,t
0,-0.217397,-0.315381,-0.349981,-0.278588,-0.213351,0
1,0.391739,0.279869,0.301570,0.311995,0.135050,1
2,-0.406473,-0.499194,-0.480291,-0.500416,0.057622,2
3,0.121576,0.122578,0.216820,0.038380,0.028551,3
4,0.261821,-0.062158,-0.234476,0.112037,-0.247819,4
...,...,...,...,...,...,...
995,-0.290144,-0.592212,-0.529248,-0.062725,-0.218547,995
996,0.316836,-0.052395,0.165297,0.337382,0.125711,996
997,-0.170996,0.098450,-0.079166,-0.544898,0.131416,997
998,0.420543,0.080840,0.188535,-0.075852,0.038052,998


In [13]:
features = df.columns.tolist()
df = preprocess_df(df, features)

In [14]:
features.remove('t')

In [ ]:
path_length = 10
n_shuffles = 1000 

ssad, shuff_sig_df, agg_shuffles = shuffled_signed_area_test(df, path_length, features, n_shuffles, plot_figures=True)

 47% (475 of 1000) |#########            | Elapsed Time: 0:07:29 ETA:   0:08:40

In [ ]:
ssad

In [ ]:
shuff_sig_df